In [0]:
dim_atleta = 'workspace.project_data_football_silver.dim_atleta'
dim_campeonato = 'workspace.project_data_football_silver.dim_campeonato'
dim_clube = 'workspace.project_data_football_silver.dim_clube'
dim_estadio = 'workspace.project_data_football_silver.dim_estadio'
dim_posicao = 'workspace.project_data_football_silver.dim_posicao'
dim_rodada = 'workspace.project_data_football_silver.dim_rodada'
fato_partida = 'workspace.project_data_football_silver.fato_partida'
fato_pontuacao = 'workspace.project_data_football_silver.fato_pontuacao'

In [0]:
spark.table(fato_partida).createOrReplaceTempView("tmp_silver_partida")
spark.table(dim_clube).createOrReplaceTempView("tmp_silver_clube")
spark.table(dim_campeonato).createOrReplaceTempView("tmp_silver_campeonato")

In [0]:
%sql
describe tmp_silver_campeonato

In [0]:
%sql
describe tmp_silver_clube

In [0]:
%sql
WITH base AS (
  -- Expande partidas (mandante + visitante)
  SELECT
    c.temporada,
    f.clube_casa_id AS clube_id,
    f.gols_casa AS gols_pro,
    f.gols_visitante AS gols_contra,
    CASE
      WHEN f.gols_casa > f.gols_visitante THEN 1
      ELSE 0
    END AS vitoria,
    CASE
      WHEN f.gols_casa < f.gols_visitante THEN 1
      ELSE 0
    END AS derrota
  FROM
    tmp_silver_partida f
      JOIN tmp_silver_campeonato c
        ON f.campeonato_id = c.campeonato_id
  UNION ALL
  SELECT
    c.temporada,
    f.clube_visitante_id AS clube_id,
    f.gols_visitante AS gols_pro,
    f.gols_casa AS gols_contra,
    CASE
      WHEN f.gols_visitante > f.gols_casa THEN 1
      ELSE 0
    END AS vitoria,
    CASE
      WHEN f.gols_visitante < f.gols_casa THEN 1
      ELSE 0
    END AS derrota
  FROM
    tmp_silver_partida f
      JOIN tmp_silver_campeonato c
        ON f.campeonato_id = c.campeonato_id
),
stats AS (
  -- Consolidação por clube e temporada
  SELECT
    temporada,
    clube_id,
    SUM(vitoria) AS vitorias,
    SUM(derrota) AS derrotas,
    SUM(gols_pro) AS gols_marcados,
    SUM(gols_contra) AS gols_sofridos,
    SUM(gols_pro - gols_contra) AS saldo
  FROM
    base
  GROUP BY
    temporada,
    clube_id
)
-- RESULTADO FINAL
-- Mais vitórias
SELECT
  s.temporada,
  'Clube com mais vitórias' AS indicador,
  CONCAT_WS(', ', SORT_ARRAY(COLLECT_SET(cl.nome_completo))) AS clubes,
  MAX(s.vitorias) AS valor
FROM
  stats s
    JOIN tmp_silver_clube cl
      ON s.clube_id = cl.clube_id
WHERE
  s.vitorias
    = (
      SELECT
        MAX(vitorias)
      FROM
        stats ss
      WHERE
        ss.temporada = s.temporada
    )
GROUP BY
  s.temporada
UNION ALL
-- Mais derrotas
SELECT
  s.temporada,
  'Clube com mais derrotas',
  CONCAT_WS(', ', SORT_ARRAY(COLLECT_SET(cl.nome_completo))),
  MAX(s.derrotas)
FROM
  stats s
    JOIN tmp_silver_clube cl
      ON s.clube_id = cl.clube_id
WHERE
  s.derrotas
    = (
      SELECT
        MAX(derrotas)
      FROM
        stats ss
      WHERE
        ss.temporada = s.temporada
    )
GROUP BY
  s.temporada
UNION ALL
-- Mais gols marcados
SELECT
  s.temporada,
  'Clube com mais gols',
  CONCAT_WS(', ', SORT_ARRAY(COLLECT_SET(cl.nome_completo))),
  MAX(s.gols_marcados)
FROM
  stats s
    JOIN tmp_silver_clube cl
      ON s.clube_id = cl.clube_id
WHERE
  s.gols_marcados
    = (
      SELECT
        MAX(gols_marcados)
      FROM
        stats ss
      WHERE
        ss.temporada = s.temporada
    )
GROUP BY
  s.temporada
UNION ALL
-- Mais gols sofridos
SELECT
  s.temporada,
  'Clube com mais gols sofridos',
  CONCAT_WS(', ', SORT_ARRAY(COLLECT_SET(cl.nome_completo))),
  MAX(s.gols_sofridos)
FROM
  stats s
    JOIN tmp_silver_clube cl
      ON s.clube_id = cl.clube_id
WHERE
  s.gols_sofridos
    = (
      SELECT
        MAX(gols_sofridos)
      FROM
        stats ss
      WHERE
        ss.temporada = s.temporada
    )
GROUP BY
  s.temporada
UNION ALL
-- Melhor saldo
SELECT
  s.temporada,
  'Melhor saldo acumulado',
  CONCAT_WS(', ', SORT_ARRAY(COLLECT_SET(cl.nome_completo))),
  MAX(s.saldo)
FROM
  stats s
    JOIN tmp_silver_clube cl
      ON s.clube_id = cl.clube_id
WHERE
  s.saldo
    = (
      SELECT
        MAX(saldo)
      FROM
        stats ss
      WHERE
        ss.temporada = s.temporada
    )
GROUP BY
  s.temporada
ORDER BY
  temporada,
  indicador;

In [0]:
df_gold_stats_clubes = spark.sql("""

WITH base AS (
  -- Expande partidas (mandante + visitante)
  SELECT
    c.temporada,
    f.clube_casa_id AS clube_id,
    f.gols_casa AS gols_pro,
    f.gols_visitante AS gols_contra,
    CASE
      WHEN f.gols_casa > f.gols_visitante THEN 1
      ELSE 0
    END AS vitoria,
    CASE
      WHEN f.gols_casa < f.gols_visitante THEN 1
      ELSE 0
    END AS derrota
  FROM
    tmp_silver_partida f
      JOIN tmp_silver_campeonato c
        ON f.campeonato_id = c.campeonato_id
  UNION ALL
  SELECT
    c.temporada,
    f.clube_visitante_id AS clube_id,
    f.gols_visitante AS gols_pro,
    f.gols_casa AS gols_contra,
    CASE
      WHEN f.gols_visitante > f.gols_casa THEN 1
      ELSE 0
    END AS vitoria,
    CASE
      WHEN f.gols_visitante < f.gols_casa THEN 1
      ELSE 0
    END AS derrota
  FROM
    tmp_silver_partida f
      JOIN tmp_silver_campeonato c
        ON f.campeonato_id = c.campeonato_id
),
stats AS (
  -- Consolidação por clube e temporada
  SELECT
    temporada,
    clube_id,
    SUM(vitoria) AS vitorias,
    SUM(derrota) AS derrotas,
    SUM(gols_pro) AS gols_marcados,
    SUM(gols_contra) AS gols_sofridos,
    SUM(gols_pro - gols_contra) AS saldo
  FROM
    base
  GROUP BY
    temporada,
    clube_id
)
-- RESULTADO FINAL
-- Mais vitórias
SELECT
  s.temporada,
  'Clube com mais vitórias' AS indicador,
  CONCAT_WS(', ', SORT_ARRAY(COLLECT_SET(cl.nome_completo))) AS clubes,
  MAX(s.vitorias) AS valor
FROM
  stats s
    JOIN tmp_silver_clube cl
      ON s.clube_id = cl.clube_id
WHERE
  s.vitorias
    = (
      SELECT
        MAX(vitorias)
      FROM
        stats ss
      WHERE
        ss.temporada = s.temporada
    )
GROUP BY
  s.temporada
UNION ALL
-- Mais derrotas
SELECT
  s.temporada,
  'Clube com mais derrotas',
  CONCAT_WS(', ', SORT_ARRAY(COLLECT_SET(cl.nome_completo))),
  MAX(s.derrotas)
FROM
  stats s
    JOIN tmp_silver_clube cl
      ON s.clube_id = cl.clube_id
WHERE
  s.derrotas
    = (
      SELECT
        MAX(derrotas)
      FROM
        stats ss
      WHERE
        ss.temporada = s.temporada
    )
GROUP BY
  s.temporada
UNION ALL
-- Mais gols marcados
SELECT
  s.temporada,
  'Clube com mais gols',
  CONCAT_WS(', ', SORT_ARRAY(COLLECT_SET(cl.nome_completo))),
  MAX(s.gols_marcados)
FROM
  stats s
    JOIN tmp_silver_clube cl
      ON s.clube_id = cl.clube_id
WHERE
  s.gols_marcados
    = (
      SELECT
        MAX(gols_marcados)
      FROM
        stats ss
      WHERE
        ss.temporada = s.temporada
    )
GROUP BY
  s.temporada
UNION ALL
-- Mais gols sofridos
SELECT
  s.temporada,
  'Clube com mais gols sofridos',
  CONCAT_WS(', ', SORT_ARRAY(COLLECT_SET(cl.nome_completo))),
  MAX(s.gols_sofridos)
FROM
  stats s
    JOIN tmp_silver_clube cl
      ON s.clube_id = cl.clube_id
WHERE
  s.gols_sofridos
    = (
      SELECT
        MAX(gols_sofridos)
      FROM
        stats ss
      WHERE
        ss.temporada = s.temporada
    )
GROUP BY
  s.temporada
UNION ALL
-- Melhor saldo
SELECT
  s.temporada,
  'Melhor saldo acumulado',
  CONCAT_WS(', ', SORT_ARRAY(COLLECT_SET(cl.nome_completo))),
  MAX(s.saldo)
FROM
  stats s
    JOIN tmp_silver_clube cl
      ON s.clube_id = cl.clube_id
WHERE
  s.saldo
    = (
      SELECT
        MAX(saldo)
      FROM
        stats ss
      WHERE
        ss.temporada = s.temporada
    )
GROUP BY
  s.temporada
ORDER BY
  temporada,
  indicador;
""")

#salvar em delta e salvar tabela
df_gold_stats_clubes.write\
    .format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .saveAsTable("workspace.project_data_football_gold.stats_clubes")


In [0]:
%sql
select * from project_data_football_gold.stats_clubes